In [6]:
%run ./pre_processing.ipynb
import pandas as pd
folds: list[pd.DataFrame]
folds

[      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
 0               7.6              0.32         0.58           16.75      0.050   
 1               5.7              0.24         0.47            6.30      0.069   
 2               7.6              0.36         0.44            8.30      0.255   
 3               7.4              0.47         0.46            2.20      0.114   
 4               9.9              0.49         0.23            2.40      0.087   
 ...             ...               ...          ...             ...        ...   
 1294            6.9              0.38         0.25            9.80      0.040   
 1295            7.3              0.23         0.37            1.80      0.032   
 1296            6.6              0.36         0.24            0.90      0.038   
 1297            6.4              0.64         0.21            1.80      0.081   
 1298            7.1              0.52         0.03            2.60      0.076   
 
       free su

In [8]:
from algos.baseline import Baseline
from sklearn.metrics import mean_absolute_error as MAE, accuracy_score as Accuracy
from imblearn.metrics import macro_averaged_mean_absolute_error as MAMAE
from algos.CEM import CEM
metrics: list[dict[str, float]] = []

for i in range(len(folds)):
  train_cv, test_cv = pd.concat(folds[j] for j in range(len(folds)) if j != i), folds[i]
  baseline = Baseline()
  baseline.fit(train_cv.drop("quality", axis=1), train_cv["quality"])
  y_pred = baseline.predict(test_cv.drop("quality", axis=1))
  metrics.append({
    "MAE": MAE(test_cv["quality"], y_pred),
    "Accuracy": Accuracy(test_cv["quality"], y_pred),
    "MAMAE": MAMAE(test_cv["quality"], y_pred),
    "CEM": CEM(test_cv["quality"], y_pred)
  })

{metric: f"{val} +- {pd.DataFrame(metrics).std()[metric]}" for metric, val in pd.DataFrame(metrics).mean().items()}

{'MAE': '0.6372121750458933 +- 0.01567434485868058',
 'Accuracy': '0.43651151773553626 +- 0.008160939902022954',
 'MAMAE': '1.6714285714285715 +- 0.09583148474999095',
 'CEM': '0.13912233473619082 +- 0.3110869976586029'}

In [ ]:
import sys
from importlib import reload
reload(sys.modules.get('algos.k_rank', sys))
from algos.k_rank import KRank
import numpy as np

metrics: list[dict[str, float]] = []
for i in range(len(folds)):
    train_cv, test_cv = pd.concat(folds[j] for j in range(len(folds)) if j != i), folds[i]
    krank = KRank()
    krank.fit(np.array(train_cv.drop("quality", axis=1).values, dtype=np.float32), train_cv["quality"].values)
    y_pred = pd.Series(krank.predict(test_cv.drop("quality", axis=1).values))
    metrics.append({
        "MAE": MAE(test_cv["quality"], y_pred),
        "Accuracy": Accuracy(test_cv["quality"], y_pred),
        "MAMAE": MAMAE(test_cv["quality"], y_pred),
        "CEM": CEM(test_cv["quality"], y_pred),
    })
{metric: f"{val} +- {pd.DataFrame(metrics).std()[metric]}" for metric, val in pd.DataFrame(metrics).mean().items()}